In [1]:
from sqlalchemy import create_engine, text
import psycopg2
import os
from hash_tournaments import hash_player_name

db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_name_2 = "ptft"


# Create engine for the first database (db_host, where tbl_player_aliases resides)
engine_db_host = create_engine(
    f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}"
)

# Create engine for the second database (ptft, where the player table resides)
engine_ptft = create_engine(
    f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name_2}"
)

# Start a session with the ptft database
with engine_ptft.connect() as connection_ptft:
    # Query players from the ptft database ordered by oldest entry first
    query = text('SELECT id, name, region, country, alias FROM player ORDER BY "createdAt" ASC')
    players = connection_ptft.execute(query).fetchall()

# Process each player
for player in players:
    player_id, player_name, player_region, player_country, player_alias = player
    new_hash = hash_player_name(player_name, region=player_region, country=player_country)
    print(player)
    all_names = [player_name] + (player_alias if player_alias else [])  # Include aliases
    print(all_names)

    # # Connect to the db_host database to check and update tbl_player_aliases
    with engine_db_host.begin() as connection_db_host:
        good_id = None
        for name in all_names:
            if not name:
                continue
            # Check if player or alias exists in tbl_player_aliases
            query_check = text("SELECT player_name, player_id, region, country FROM tbl_player_aliases WHERE player_name = :name")
            result = connection_db_host.execute(query_check, {"name": name}).fetchall()

            if len(result) > 1:
                detail_check = text("SELECT player_name, player_id, region, country FROM tbl_player_aliases WHERE player_name = :name AND region = :region AND country = :country ")
                result = connection_db_host.execute(detail_check, {"name": name, "region":player_region, "country":player_country}).fetchone()
            else:
                result = connection_db_host.execute(query_check, {"name": name}).fetchone()
                

            print(result)
            if result:
                alias_name, alias_id, alias_region, alias_country = result
                if alias_id:
                    good_id = alias_id


                if not alias_region or not alias_country:
                    # Update region/country if empty
                    query_update = text("""
                        UPDATE tbl_player_aliases
                        SET region = :region, country = :country
                        WHERE player_name = :name
                    """)
                    connection_db_host.execute(query_update, {
                        "region": player_region,
                        "country": player_country,
                        "name": alias_name
                    })
                elif alias_region != player_region or alias_country != player_country:
                    # Insert a new row if the region or country differ
                    query_insert = text("""
                        INSERT INTO tbl_player_aliases (player_name, player_id, region, country)
                        VALUES (:name, :player_id, :region, :country)
                    """)
                    connection_db_host.execute(query_insert, {
                        "name": name,
                        "player_id": new_hash if good_id is None or alias_region != player_region or alias_country != player_country else good_id,
                        "region": player_region,
                        "country": player_country
                    })
            else:
                # Insert new entry if the name or alias doesn't exist in tbl_player_aliases
                query_insert = text("""
                    INSERT INTO tbl_player_aliases (player_name, player_id, region, country)
                    VALUES (:name, :player_id, :region, :country)
                """)
                connection_db_host.execute(query_insert, {
                    "name": name,
                    "player_id": new_hash,
                    "region": player_region,
                    "country": player_country
                })

(77, 'vclf', 'BR', 'BRA', ['RED vclf', 'Yutaka Hoshino', 'Red vclf', 'did my best'])
['vclf', 'RED vclf', 'Yutaka Hoshino', 'Red vclf', 'did my best']
('vclf', 79038099, 'BR', 'BRA')
('RED vclf', 79038099, 'BR', 'BRA')
('Yutaka Hoshino', 79038099, 'BR', 'BRA')
('Red vclf', 79038099, 'BR', 'BRA')
('did my best', 79038099, 'BR', 'BRA')
(104, 'Slooper', 'BR', 'BRA', ['INTZ Slooper'])
['Slooper', 'INTZ Slooper']
('Slooper', 95265227, 'BR', 'BRA')
('INTZ Slooper', 95265227, 'BR', 'BRA')
(466, 'LilTop', 'NA', 'USA', [])
['LilTop']
('LilTop', 5072863, 'NA', 'USA')
(376, 'KevChen', 'NA', 'USA', [])
['KevChen']
('KevChen', 18794348, 'NA', 'USA')
(557, 'P1va', 'SEA', 'VNM', [])
['P1va']
('P1va', 93521172, 'SEA', 'VNM')
(560, 'Qkt', 'SEA', 'TWN', [])
['Qkt']
('Qkt', 83606271, 'SEA', 'TWN')
(380, 'Bradini', 'NA', 'USA', [])
['Bradini']
('Bradini', 57116891, 'NA', 'USA')
(68, 'Un33d', 'EMEA', 'FRA', ['Um33d', 'AEG Un33d'])
['Un33d', 'Um33d', 'AEG Un33d']
('Un33d', 27141613, 'EMEA', 'FRA')
('Um33d',

In [6]:
# puts all the region/countries down for ids that exist
from sqlalchemy import create_engine, text
import os

# Setup database connection
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")

# Create engine for the database
engine_db_host = create_engine(
    f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}"
)

# Query for rows with NULL region or country
with engine_db_host.begin() as connection_db_host:
    # Step 1: Fetch rows with NULL region or country
    query_nulls = text("""
        SELECT player_name, player_id, region, country
        FROM tbl_player_aliases
        WHERE region IS NULL OR country IS NULL
    """)
    null_results = connection_db_host.execute(query_nulls).fetchall()

    # Process each row with nulls
    for row in null_results:
        player_name, player_id, region, country = row
        print(f"Processing: player_id={player_id}, player_name={player_name}, region={region}, country={country}")

        # Step 2: Look for rows with the same player_id that have non-null region and country
        query_non_nulls = text("""
            SELECT region, country
            FROM tbl_player_aliases
            WHERE player_id = :player_id
            AND region IS NOT NULL AND country IS NOT NULL
        """)
        non_null_result = connection_db_host.execute(query_non_nulls, {"player_id": player_id}).fetchone()

        # Step 3: If a row with the same player_id and non-null region/country exists, update the current row
        if non_null_result:
            non_null_region, non_null_country = non_null_result
            print(f"Found non-null values for player_id={player_id}: region={non_null_region}, country={non_null_country}")

            # Step 4: Update the row with the non-null region/country
            query_update = text("""
                UPDATE tbl_player_aliases
                SET region = :region, country = :country
                WHERE player_id = :player_id AND (region IS NULL OR country IS NULL)
            """)
            connection_db_host.execute(query_update, {
                "region": non_null_region,
                "country": non_null_country,
                "player_id": player_id
            })
            print(f"Updated: player_id={player_id} with region={non_null_region}, country={non_null_country}")
        else:
            print(f"No non-null region/country found for player_id={player_id}")


Processing: player_id=45665142, player_name=Riverx, region=None, country=None
No non-null region/country found for player_id=45665142
Processing: player_id=13945082, player_name=mogumoguyummy, region=None, country=None
No non-null region/country found for player_id=13945082
Processing: player_id=62741729, player_name=Carelessd, region=None, country=None
No non-null region/country found for player_id=62741729
Processing: player_id=63079728, player_name=SealkunMbappe, region=None, country=None
Found non-null values for player_id=63079728: region=LA, country=COL
Updated: player_id=63079728 with region=LA, country=COL
Processing: player_id=63079728, player_name=SealKun Mbappe, region=None, country=None
Found non-null values for player_id=63079728: region=LA, country=COL
Updated: player_id=63079728 with region=LA, country=COL
Processing: player_id=27993639, player_name=socks, region=None, country=None
Found non-null values for player_id=27993639: region=NA, country=USA
Updated: player_id=27

In [ ]:
from sqlalchemy import create_engine, text
import psycopg2
import os
from hash_tournaments import hash_player_name

db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_name_2 = "ptft"


engine_ptft = create_engine(
    f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name_2}"
)

# Start a session with the ptft database
with engine_ptft.connect() as connection_ptft:
    # Query players from the ptft database ordered by oldest entry first
    query = text('SELECT id, name, region, country, alias FROM player ORDER BY "createdAt" ASC')
    players = connection_ptft.execute(query).fetchall()